In [37]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [38]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot

In [39]:
init_notebook_mode(connected=True)

In [40]:
database_url = os.getenv('DATABASE_URL')
engine = create_engine(database_url)

In [41]:
query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
"""

In [42]:
df_tables = pd.read_sql(query, engine)
df_tables = df_tables[df_tables['table_name'].str.contains('SUBE')]
df_tables

,table_name
2,Transacciones_SUBE_2022
3,Transacciones_SUBE_2023
4,Transacciones_SUBE_2021
5,Transacciones_SUBE_2020


In [43]:
df_Transacciones_SUBE = pd.DataFrame()
for table in df_tables['table_name']:
    print(table)
    df = pd.read_sql(f'SELECT * FROM "public"."{table}"', engine)
    df_Transacciones_SUBE = pd.concat([df_Transacciones_SUBE, df], ignore_index=True)
engine.dispose()

Transacciones_SUBE_2022
Transacciones_SUBE_2023
Transacciones_SUBE_2021
Transacciones_SUBE_2020


In [44]:
df_Transacciones_SUBE.head()

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR,Anio,Mes,Periodo
0,2022-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,10,NO,2022,01,2022-01
1,2022-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,19,NO,2022,01,2022-01
2,2022-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1418,NO,2022,01,2022-01
3,2022-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,675,NO,2022,01,2022-01
4,2022-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1906,NO,2022,01,2022-01


In [45]:
df_Transacciones_SUBE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1579839 entries, 0 to 1579838
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   DIA_TRANSPORTE   1579839 non-null  object
 1   NOMBRE_EMPRESA   1579839 non-null  object
 2   LINEA            1579839 non-null  object
 3   AMBA             1579839 non-null  object
 4   TIPO_TRANSPORTE  1579839 non-null  object
 5   JURISDICCION     1569691 non-null  object
 6   PROVINCIA        1569691 non-null  object
 7   MUNICIPIO        1569691 non-null  object
 8   CANTIDAD         1579839 non-null  int64 
 9   DATO_PRELIMINAR  1579839 non-null  object
 10  Anio             1579839 non-null  object
 11  Mes              1579839 non-null  object
 12  Periodo          1579839 non-null  object
dtypes: int64(1), object(12)
memory usage: 156.7+ MB


In [46]:
df_Transacciones_SUBE.isnull().sum()

DIA_TRANSPORTE         0
NOMBRE_EMPRESA         0
LINEA                  0
AMBA                   0
TIPO_TRANSPORTE        0
JURISDICCION       10148
PROVINCIA          10148
MUNICIPIO          10148
CANTIDAD               0
DATO_PRELIMINAR        0
Anio                   0
Mes                    0
Periodo                0
dtype: int64

¿En que periodos se registraron la mayor cantidad de transacciones de SUBE?

In [47]:
df_Transacciones_SUBE_periodo = df_Transacciones_SUBE.groupby('Periodo').size().reset_index(name='count').nlargest(10, 'count')
df_Transacciones_SUBE_periodo

,Periodo,count
43,2023-08,41327
38,2023-03,40524
40,2023-05,40349
42,2023-07,39643
41,2023-06,39054
39,2023-04,38801
31,2022-08,38214
26,2022-03,38093
35,2022-12,37611
44,2023-09,37477


In [48]:
fig = px.bar(df_Transacciones_SUBE_periodo, x='Periodo', y='count')
iplot(fig)

¿En que año se registro la mayor cantidad de transacciones de SUBE?

In [49]:
df_Transacciones_SUBE_por_anio = df_Transacciones_SUBE.groupby('Anio').size().reset_index(name='count')
df_Transacciones_SUBE_por_anio

,Anio,count
0,2020,356628
1,2021,410644
2,2022,439337
3,2023,373230


In [50]:
fig = px.pie(df_Transacciones_SUBE_por_anio, values='count', names='Anio', title='Transacciones SUBE por año')
iplot(fig)